In [1]:
#import Pkg; 
#Pkg.add("ITensors")
#Pkg.add("Combinatorics")

In [2]:
using ITensors
using Combinatorics
using IterTools: product
using CSV, DataFrames

In [3]:
CCNOT_mat = zeros(2,2,2,2,2,2)
for i in 1:2
    for j in 1:2
        for k in 1:2
            CCNOT_mat[i,j,k,i,j,k] = 1.0
        end
    end
end

CCNOT_mat[2,2,2,2,2,2] = 0.
CCNOT_mat[2,2,1,2,2,1] = 0.
CCNOT_mat[2,2,1,2,2,2] = 1.
CCNOT_mat[2,2,2,2,2,1] = 1.

1.0

In [4]:

i1 = Index(2, "i1")
i2 = Index(2, "i2")
i3 = Index(2, "i3")
j1 = Index(2, "j1")
j2 = Index(2, "j2")
j3 = Index(2, "j3")
k1 = Index(2, "k1")
k2 = Index(2, "k2")
k3 = Index(2, "k3")
l1 = Index(2, "l1")
l2 = Index(2, "l2")
l3 = Index(2, "l3")

(dim=2|id=64|"l3")

In [5]:
function tags_l(t)
    return [tags(i) for i in t]
end

tags_l (generic function with 1 method)

In [6]:
function true_val(input_ind,dag_input_ind,dag_input_val,con_input_ind,con_input_val,output_ind,dag_output_ind)
    A = ITensor(CCNOT_mat,i1,i2,i3,j1,j2,j3)
    B = ITensor(CCNOT_mat,k1,k2,k3,l1,l2,l3)

    con_list = [delta(con_input_ind[i],con_input_val[i]) for i in 1:length(con_input_ind)]
    dagin_list = [onehot(dag_input_ind[i]=>dag_input_val[i]) for i in 1:length(dag_input_ind)]
    dagout_list = [ITensor([1. 1.],i) for i in dag_output_ind]

    # Contract the tensors over the common indices
    # and visualize the results
    AB = A * B * prod(dagin_list) * prod(con_list)  * prod(dagout_list) *onehot(output_ind=>2);
    AB = Array(AB,input_ind[1],input_ind[2]);
    AB = reshape(AB,4);
    return transpose(AB)
end

true_val (generic function with 1 method)

In [7]:
df = DataFrame(input_ind=[], dag_input_ind=[], dag_input_val=[], con_input_ind=[], con_input_val=[], output_ind=[], dag_output_ind=[], tv=[])

for con_input_ind in collect(combinations([k1,k2,k3]))
    for subset in collect(combinations([j1,j2,j3],length(con_input_ind)))
        for con_input_val in collect(permutations(subset)) 
            for input_ind in collect(combinations(setdiff([i1,i2,i3,k1,k2,k3],con_input_ind),2))
                dag_input_ind = setdiff(setdiff([i1,i2,i3,k1,k2,k3],input_ind),con_input_ind)
                for dag_input_val in [collect(t) for t in product(repeat([[1, 2]], length(dag_input_ind))...)]
                    for output_ind in setdiff([j1,j2,j3,l1,l2,l3],con_input_val)
                        dag_output_ind = setdiff(setdiff([j1,j2,j3,l1,l2,l3],[output_ind]),con_input_val)
                        tv = true_val(input_ind,dag_input_ind,dag_input_val,con_input_ind,con_input_val,output_ind,dag_output_ind);
                        push!(df, (tags_l(input_ind), tags_l(dag_input_ind), dag_input_val, tags_l(con_input_ind), tags_l(con_input_val), tags(output_ind), tags_l(dag_output_ind), tv))
                        #@show tags_l(input_ind)
                    end
                end
            end
        end
    end
end

# Write the DataFrame to a CSV file
CSV.write("output_results.csv", df)

println("Data written to output_results.csv")

Data written to output_results.csv


In [8]:

input_ind = [i1,i2]
dag_input_ind = [i3]
dag_input_val = [1]
con_input_ind = [k1,k2,k3]
con_input_val = [j1,j2,j3]

ouput_ind = l3
dag_output_ind = [l1,l2]

true_val(input_ind,dag_input_ind,dag_input_val,con_input_ind,con_input_val,ouput_ind,dag_output_ind)

1×4 transpose(::Vector{Float64}) with eltype Float64:
 0.0  0.0  0.0  0.0

In [9]:
string(tags_l(dag_input_ind))

"TagSet[\"i3\"]"